# Topsis on Text Classifications Model

DataSet Link: https://huggingface.co/datasets/jakeazcona/short-text-labeled-emotion-classification

**Name:** Saransh

**Roll Number:** 102103077

**Group:** 3CO3


In [ ]:
!pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("jakeazcona/short-text-labeled-emotion-classification")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val_iron split: 0 examples [00:00, ? examples/s]

Generating val_neg split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sample', 'label'],
        num_rows: 24032
    })
    test: Dataset({
        features: ['sample', 'label'],
        num_rows: 6008
    })
    val_iron: Dataset({
        features: ['sample', 'label'],
        num_rows: 28
    })
    val_neg: Dataset({
        features: ['sample', 'label'],
        num_rows: 24
    })
})

In [ ]:
dataset["train"][0]

{'sample': 'i tend to be a window shopper when im alone because theres always going to be a self imposed limit of one or two when im feeling naughty',
 'label': 3}

In [ ]:
train_data = dataset["train"]

In [ ]:
train_data

Dataset({
    features: ['sample', 'label'],
    num_rows: 24032
})

In [ ]:
import pandas as pd

In [ ]:
# Convert to Pandas DataFrame
train_df = pd.DataFrame(train_data)

In [ ]:
train_df.head()

,sample,label
0,i tend to be a window shopper when im alone be...,3
1,i will hopefully be able to feel less inhibite...,5
2,i feel very fond of my pinky kids,3
3,i feel like i had so much to write then got di...,0
4,i knew i was going to look at the mess and fee...,5


In [ ]:
train_df.describe()

,label
count,24032.000000
mean,2.603196
std,1.729178
min,0.000000
25%,1.000000
50%,2.000000
75%,4.000000
max,5.000000


In [ ]:
# Assuming your original DataFrame is named train_df
# Create separate DataFrames for label 0 and label 1
label_0_df = train_df[train_df['label'] == 0].head(1500)
label_1_df = train_df[train_df['label'] == 1].head(1500)
label_2_df = train_df[train_df['label'] == 2].head(1500)
label_3_df = train_df[train_df['label'] == 3].head(1500)
label_4_df = train_df[train_df['label'] == 4].head(1500)
label_5_df = train_df[train_df['label'] == 5].head(6500)

# Concatenate the two DataFrames to create the new DataFrame
train_df = pd.concat([label_0_df, label_1_df,label_2_df, label_3_df,label_4_df, label_5_df])

# Shuffle the new DataFrame to randomize the order of rows
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [ ]:
#Preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer, DistilBertForSequenceClassification, DistilBertTokenizer, RobertaForSequenceClassification, RobertaTokenizer, XLNetForSequenceClassification, XLNetTokenizer, AlbertForSequenceClassification, AlbertTokenizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
train_df['sample'] = train_df['sample'].apply(preprocess_text)

In [ ]:
train_df.head()

,sample,label
0,still feeling gloomy,5
1,youre back youre still broken,5
2,oh know ’ going,4
3,5x5 hurt eye,5
4,need find way get yet feel hopeless,5


In [ ]:
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:

model_names = ["bert-base-uncased", "distilbert-base-uncased", "roberta-base", "albert-base-v2", "xlnet-base-cased"]

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# Dictionary to store results
results_dict = {}

In [ ]:
import warnings
warnings.filterwarnings("ignore")
for model_name in model_names:
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Tokenize and prepare data
    train_texts = train_df['sample'].tolist()
    train_labels = train_df['label'].tolist()

    # Use smaller batches for tokenization
    batch_size = 32
    num_batches = len(train_texts) // batch_size + (len(train_texts) % batch_size != 0)

    predictions = []

    # Use tqdm for a progress bar
    for i in tqdm(range(num_batches), desc=f"Training {model_name}"):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        batch_texts = train_texts[start_idx:end_idx]
        batch_labels = train_labels[start_idx:end_idx]

        tokenized_data = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")

        with torch.no_grad():
            logits = model(**tokenized_data).logits

        batch_predictions = torch.argmax(logits, dim=1).tolist()
        predictions.extend(batch_predictions)

    # Evaluate the model on the test set
    test_texts = test_df['sample'].tolist()
    test_labels = test_df['label'].tolist()

    tokenized_test_data = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")

    with torch.no_grad():
        logits = model(**tokenized_test_data).logits

    predictions = torch.argmax(logits, dim=1).tolist()

    # Store classification report and other metrics in the dictionary
    accuracy = accuracy_score(test_labels, predictions)
    precision_weighted = precision_score(test_labels, predictions, average='weighted')
    recall_weighted = recall_score(test_labels, predictions, average='weighted')
    f1_weighted = f1_score(test_labels, predictions, average='weighted')
    results_dict[model_name] = {
        "accuracy": accuracy,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
    }
    print(results_dict)
    # Print metrics
    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (Weighted): {precision_weighted:.4f}")
    print(f"Recall (Weighted): {recall_weighted:.4f}")
    print(f"F1 Score (Weighted): {f1_weighted:.4f}")
    print("=" * 50)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training xlnet-base-cased: 100%|██████████| 336/336 [17:33<00:00,  3.14s/it]


{'bert-base-uncased': {'accuracy': 0.10577281191806331, 'precision_weighted': 0.025238499751268604, 'recall_weighted': 0.10577281191806331, 'f1_weighted': 0.022778378101137583}, 'distilbert-base-uncased': {'accuracy': 0.11396648044692738, 'precision_weighted': 0.02378788571055139, 'recall_weighted': 0.11396648044692738, 'f1_weighted': 0.039205896829196306}, 'roberta-base': {'accuracy': 0.09459962756052141, 'precision_weighted': 0.017627308725068378, 'recall_weighted': 0.09459962756052141, 'f1_weighted': 0.028962202905588534}, 'albert-base-v2': {'accuracy': 0.10204841713221602, 'precision_weighted': 0.021694006872754287, 'recall_weighted': 0.10204841713221602, 'f1_weighted': 0.032770042315949564}, 'xlnet-base-cased': {'accuracy': 0.10093109869646183, 'precision_weighted': 0.020183012045490018, 'recall_weighted': 0.10093109869646183, 'f1_weighted': 0.024880203262319902}}
Model: xlnet-base-cased
Accuracy: 0.1009
Precision (Weighted): 0.0202
Recall (Weighted): 0.1009
F1 Score (Weighted): 0

In [ ]:
results_dict

{'bert-base-uncased': {'accuracy': 0.10577281191806331,
  'precision_weighted': 0.025238499751268604,
  'recall_weighted': 0.10577281191806331,
  'f1_weighted': 0.022778378101137583},
 'distilbert-base-uncased': {'accuracy': 0.11396648044692738,
  'precision_weighted': 0.02378788571055139,
  'recall_weighted': 0.11396648044692738,
  'f1_weighted': 0.039205896829196306},
 'roberta-base': {'accuracy': 0.09459962756052141,
  'precision_weighted': 0.017627308725068378,
  'recall_weighted': 0.09459962756052141,
  'f1_weighted': 0.028962202905588534},
 'albert-base-v2': {'accuracy': 0.10204841713221602,
  'precision_weighted': 0.021694006872754287,
  'recall_weighted': 0.10204841713221602,
  'f1_weighted': 0.032770042315949564},
 'xlnet-base-cased': {'accuracy': 0.10093109869646183,
  'precision_weighted': 0.020183012045490018,
  'recall_weighted': 0.10093109869646183,
  'f1_weighted': 0.024880203262319902}}

In [ ]:
results_df = pd.DataFrame(results_dict).T
results_df.to_csv('Final_results.csv')
print(results_df)

                         accuracy  precision_weighted  recall_weighted  \
bert-base-uncased        0.105773            0.025238         0.105773   
distilbert-base-uncased  0.113966            0.023788         0.113966   
roberta-base             0.094600            0.017627         0.094600   
albert-base-v2           0.102048            0.021694         0.102048   
xlnet-base-cased         0.100931            0.020183         0.100931   

                         f1_weighted  
bert-base-uncased           0.022778  
distilbert-base-uncased     0.039206  
roberta-base                0.028962  
albert-base-v2              0.032770  
xlnet-base-cased            0.024880  


In [ ]:
!pip install TOPSIS-Rajneesh-102283007


ModuleNotFoundError: No module named 'topsis'

In [ ]:
!.\topisis.py Final_results.csv "1,1,1,1" "+,+,+,+" topsis.csv

In [ ]:
import pandas as pd

In [38]:
FinalAnswer = pd.read_csv('topsis.csv')

In [39]:
FinalAnswer

,Unnamed: 0,accuracy,precision_weighted,recall_weighted,f1_weighted,Performance,rank
0,bert-base-uncased,0.105773,0.025238,0.105773,0.022778,0.41,3
1,distilbert-base-uncased,0.113966,0.023788,0.113966,0.039206,0.91,1
2,roberta-base,0.094600,0.017627,0.094600,0.028962,0.27,4
3,albert-base-v2,0.102048,0.021694,0.102048,0.032770,0.56,2
4,xlnet-base-cased,0.100931,0.020183,0.100931,0.024880,0.22,5


# So I can conclude that distilbert-base-uncased pretrained model works better on text classification for our dataset